In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Distance matrix between Durban and other cities (in kilometers)
    data['distance_matrix'] = [
        [0, 568, 634, 569, 392],  # Durban
        [568, 0, 121, 595, 100],  # Johannesburg
        [634, 121, 0, 712, 216],  # Pretoria
        [569, 595, 712, 0, 497],  # Cape Town
        [392, 100, 216, 497, 0],  # Bloemfontein
    ]
    data['num_vehicles'] = 1
    data['depot'] = 2  # Starting at Durban
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} kilometers'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    print('Route distance: {} kilometers'.format(route_distance))

def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Set up search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)

if __name__ == '__main__':
    main()


Objective: 1893 kilometers
Route:
 2 -> 1 -> 3 -> 0 -> 4 -> 2

Route distance: 1893 kilometers


In [7]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    # Distance matrix between Durban and other cities (in kilometers)
    data['distance_matrix'] = [
        [0, 568, 634, 569, 392],  # Durban
        [568, 0, 121, 595, 100],  # Johannesburg
        [634, 121, 0, 712, 216],  # Pretoria
        [569, 595, 712, 0, 497],  # Cape Town
        [392, 100, 216, 497, 0],  # Bloemfontein
    ]
    data['demands'] = [0, 1, 1, 2, 4]  # Cargo demands in each city
    data['vehicle_capacities'] = [5]    # Vehicle capacity
    data['num_vehicles'] = 1
    data['depot'] = 0  # Starting at Durban
    return data


In [4]:
def main():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Set up search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)

if __name__ == '__main__':
    main()


In [5]:
import gradio as gr

!pip install gradio


def solve_vrp():
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Set up search parameters.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        manager, routing, solution = manager, routing, solution
        index = routing.Start(0)
        plan_output = 'Route:\n'
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} ->'.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        plan_output += ' {}\n'.format(manager.IndexToNode(index))
        return plan_output + 'Route distance: {} kilometers'.format(route_distance)
    else:
        return "No solution found!"

iface = gr.Interface(fn=solve_vrp, inputs=[], outputs="text")
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Generate dummy data
np.random.seed(42)
data_size = 1000
sensor_data = np.random.rand(data_size, 5)  # 5 sensor readings
maintenance_events = np.random.randint(2, size=data_size)  # 0 or 1 indicating maintenance needed or not

# Create a DataFrame
df = pd.DataFrame(sensor_data, columns=[f'sensor_{i}' for i in range(1, 6)])
df['maintenance_needed'] = maintenance_events

# Split the data into training and testing sets
X = df.drop('maintenance_needed', axis=1)
y = df['maintenance_needed']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))

# Function to predict maintenance need based on new sensor data
def predict_maintenance(sensor_readings):
    return model.predict([sensor_readings])

# Example usage
new_sensor_readings = np.random.rand(5)  # New sensor readings
print(f"New sensor readings: {new_sensor_readings}")
print(f"Maintenance needed: {predict_maintenance(new_sensor_readings)[0]}")

              precision    recall  f1-score   support

           0       0.45      0.37      0.41        99
           1       0.47      0.55      0.51       101

    accuracy                           0.47       200
   macro avg       0.46      0.46      0.46       200
weighted avg       0.46      0.47      0.46       200

New sensor readings: [0.51874785 0.05116551 0.23285856 0.49200173 0.31023528]
Maintenance needed: 1


c:\Users\craigparker\Anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
